In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm

#### 농가 데이터 분할된 파일 합치기

In [3]:
# 파일이 존재하는 경로 불러오기 및 리스트
path = '../data'
file_lst = os.listdir(path)# 폴더 내 파일 리스트
file_lst

['2022-12.xlsx',
 '2023-01.xlsx',
 '2023-02.xlsx',
 '2023-03.xlsx',
 '2023-04.xlsx',
 '2023-05.xlsx',
 '2023-06.xlsx',
 '2023-07.xlsx',
 '2023-08.xlsx']

In [4]:
df = pd.DataFrame()
for i in file_lst: # i에 파일명 담기
    data_3 = pd.read_excel(path +'/'+i) # 파일 불러오기
    df = pd.concat([df, data_3])
    
df = df.reset_index(drop = True)

df.head(2)

,개월령,생시체중,근내평균,도체범위근내평균,체고,체장,형매도축수,형매도체평균,형매근내평균,형매근내평균가산,6개월내출산,농가근내평균,농가근내평균가산,근내EPD,근내,점수
0,46,644.64,3.34,3.23,131.4,158.9,NaN,NaN,NaN,NaN,N,4.16,-1.05,0.18,6,4
1,46,633.93,4.59,4.52,129.7,148.4,NaN,NaN,NaN,NaN,Y,4.16,-1.05,0.17,4,3


In [5]:
a = df.columns
print(df.shape)

(92358, 16)


#### 우시장 암소 데이터 불러오기

In [6]:
df1 = pd.read_excel('../2023-01-2023-09-우시장.xlsx') 
df1.columns

Index(['개월령', '생시체중', '근내평균', '도체범위근내평균', '체고', '체장', '형매도축수', '형매도체평균',
       '형매근내평균', '형매근내평균가산', '6개월내출산', '농가근내평균', '농가근내평균가산', '근내EPD', '근내',
       '도축등급점수'],
      dtype='object')

In [7]:
df1_copy = df1.copy()
df1_copy.columns = a
df1_copy.head()

,개월령,생시체중,근내평균,도체범위근내평균,체고,체장,형매도축수,형매도체평균,형매근내평균,형매근내평균가산,6개월내출산,농가근내평균,농가근내평균가산,근내EPD,근내,점수
0,37,620,4.27,4.32,130.3,153.2,NaN,NaN,NaN,NaN,Y,4.75,-0.20,0.39,2,2
1,34,608,5.49,4.95,130.3,157.4,NaN,NaN,NaN,NaN,Y,4.75,-0.20,0.45,4,3
2,35,576,3.77,2.33,133.3,159.9,NaN,NaN,NaN,NaN,Y,4.75,-0.20,-0.01,3,2
3,37,600,5.16,5.46,129.4,148.4,NaN,NaN,NaN,NaN,N,NaN,NaN,0.29,8,5
4,60,563,4.38,3.90,128.0,148.5,2.0,429.5,4.0,-1.48,N,5.56,1.31,0.26,3,2


In [8]:
df1_copy.shape

(7540, 16)

In [9]:
# full 데이터 셋 내보내기
# df.to_csv('../data/full.csv',index=False)

In [10]:
print(df['농가근내평균가산'][:5])
cow = df[df['농가근내평균가산']!='\\N'].copy()
cow['농가근내평균가산'] = cow['농가근내평균가산'].astype('float64')
cow['농가근내평균가산'].head()

0   -1.05
1   -1.05
2    0.53
3   -0.91
4   -1.35
Name: 농가근내평균가산, dtype: object


0   -1.05
1   -1.05
2    0.53
3   -0.91
4   -1.35
Name: 농가근내평균가산, dtype: float64

##### 데이터 합치기

In [11]:
cow['농가구분'] = '농가소'
df1_copy['농가구분'] = '우시장소'

In [12]:
cow['농가구분']

0        농가소
1        농가소
2        농가소
3        농가소
4        농가소
        ... 
92353    농가소
92354    농가소
92355    농가소
92356    농가소
92357    농가소
Name: 농가구분, Length: 92315, dtype: object

In [13]:
df_merge = pd.concat([cow, df1_copy])
print(f'농가소 마릿수:{cow.shape[0]}');print(f'우시장 소 마릿수:{df1_copy.shape[0]}')
print(f'합친 거 마릿수 확인:{cow.shape[0] + df1_copy.shape[0]}')
print(f'합친 거 마릿수:{df_merge.shape[0]}')


농가소 마릿수:92315
우시장 소 마릿수:7540
합친 거 마릿수 확인:99855
합친 거 마릿수:99855


In [14]:
df_merge = df_merge.reset_index(drop=True)
# df_merge.drop(columns='level_0', axis=1, inplace=True)
print(df_merge.head(1))
print(df_merge.tail(1))

   개월령    생시체중  근내평균  도체범위근내평균     체고     체장  형매도축수  형매도체평균  형매근내평균  형매근내평균가산  \
0   46  644.64  3.34      3.23  131.4  158.9    NaN     NaN     NaN       NaN   

  6개월내출산  농가근내평균  농가근내평균가산  근내EPD  근내  점수 농가구분  
0      N    4.16     -1.05   0.18   6   4  농가소  
       개월령   생시체중  근내평균  도체범위근내평균     체고     체장  형매도축수  형매도체평균  형매근내평균  \
99854   30  606.0  5.66      4.98  128.6  149.9    NaN     NaN     NaN   

       형매근내평균가산 6개월내출산  농가근내평균  농가근내평균가산  근내EPD  근내  점수  농가구분  
99854       NaN      N     NaN       NaN   0.06   5   3  우시장소  


In [15]:
df_merge.to_csv('../data_2/merge_data.csv')

In [21]:
df_merge['TARGET1'] = df_merge['점수'].apply(lambda x: 1 if 5 == x else 0 if 4 >= x else 0)
df_merge[['점수','TARGET1']].head(3)

,점수,TARGET1
0,4,0
1,3,0
2,5,1


In [22]:
df_merge.to_csv('../data_2/merge_data.csv')

##### 로지스틱 반복문

In [ ]:
for i in cow.columns:
    model = sm.Logit.from_formula('점수 ~ cow[i]', cow).fit()
    print(f'독립변수 이름: {i}')
    print(model.summary())
    print('============='*3)
    print(f'cow:{model.aic}')
    print('\n')
    print(np.exp(model.params))